In [284]:
import numpy as np
import pandas as pd
import networkx as nx

%matplotlib notebook

def splitDataFrameList(df,target_column,separator):
	''' df = dataframe to split,
	target_column = the column containing the values to split
	separator = the symbol used to perform the split
	returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
	The values in the other columns are duplicated across the newly divided rows.
	'''
	def splitListToRows(row,row_accumulator,target_column,separator):
		split_row = row[target_column].split(separator)
		for s in split_row:
			new_row = row.to_dict()
			new_row[target_column] = s
			row_accumulator.append(new_row)
	new_rows = []
	df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
	new_df = pd.DataFrame(new_rows)
	return new_df

In [510]:
df = pd.read_csv('../reading_and_cleaning/cleaned_podcasts.csv', sep='\t', index_col=0)
podcast_info = pd.read_csv('../reading_and_cleaning/meta_podcast_info.csv', sep='\t', index_col=0)
# df = df.replace(r'', np.nan, regex=True)
# df = df[pd.notnull(df['guests'])]
# split_hosts = splitDataFrameList(df, 'hosts', ', ')

# for index, row in split_hosts.iterrows():
#     if(row['hosts'] == row['guests']):
#         split_hosts.drop(index=index, inplace=True)

G1 = nx.from_pandas_dataframe(split_hosts, 'guests', 'hosts', edge_attr=['date', 'duration', 'podcast'], create_using=nx.MultiDiGraph())
df

,date,duration,episode,guests,hosts,podcast,title,podcast_id
0,2018-03-20 22:29:51,10603,1094,Brian Redban,Joe Rogan,The Joe Rogan Experience,#1094 - Brian Redban,0
1,2018-03-19 23:20:58,11990,1093,Owen Benjamin,Joe Rogan,The Joe Rogan Experience,#1093 - Owen Benjamin & Kurt Metzger,0
2,2018-03-19 23:20:58,11990,1093,Kurt Metzger,Joe Rogan,The Joe Rogan Experience,#1093 - Owen Benjamin & Kurt Metzger,0
3,2018-03-15 00:26:37,7725,1092,Mary Lynn Rajskub,Joe Rogan,The Joe Rogan Experience,#1092 - Mary Lynn Rajskub,0
4,2018-03-14 23:39:03,9406,1091,Daniele Bolelli,Joe Rogan,The Joe Rogan Experience,#1091 - Daniele Bolelli,0
5,2018-03-13 23:22:42,11795,1090,Andrew Santino,Joe Rogan,The Joe Rogan Experience,#1090 - Andrew Santino,0
6,2018-03-08 02:07:37,13856,1089,John Dudley,Joe Rogan,The Joe Rogan Experience,#1089 - John Dudley,0
7,2018-03-07 03:06:07,11805,1088,Bryan Callen,Joe Rogan,The Joe Rogan Experience,#1088 - Bryan Callen,0
8,2018-03-05 23:37:15,10793,1087,Sturgill Simpson,Joe Rogan,The Joe Rogan Experience,#1087 - Sturgill Simpson,0
9,2018-03-01 00:54:02,11608,1086,Rory Albanese,Joe Rogan,The Joe Rogan Experience,#1086 - Rory Albanese,0


In [ ]:
len(G1.edges())

In [ ]:
import ast

for index1, row1 in podcast_info.iterrows():
    df1 = df[df['podcast'] == row1['Podcast Name']].copy()
    #print(df1.is_copy)
    hosts = ast.literal_eval(row1['Hosts'])
    #print(hosts[0])
    for host in hosts:
        #print(host)
        for index2, row2 in df1.iterrows():
            if(row2['guests'] == host):
                df1.drop(index=index2, inplace=True)
                #print('dropping', row2['guests'])
        #print(host)
    guest_durations1 = df1.groupby(['guests'])['duration'].sum()
    guest_durations1.sort_values(ascending=False, inplace=True)
    filename = 'top_guests/' + row1['Podcast Name'] + '.csv'
    guest_durations1.to_csv(filename)
    print(row1['Podcast Name'], ' - ', guest_durations1.index[0], guest_durations1.values[0])

# joe_rogan = split_hosts[split_hosts['hosts'] == 'Joe Rogan']
# #guest_durations = joe_rogan['duration'].groupby(joe_rogan['guests']).sum()
# guest_durations = joe_rogan.groupby(['guests'])['duration'].sum()

# guest_durations.sort_values(ascending=False, inplace=True)
# guest_durations.index[0]

In [ ]:
hosts = ['Brendan Schaub']
for host in hosts:
    print(host)

In [74]:
guest_durations = split_hosts.groupby(['hosts', 'guests'])['duration'].sum()
guest_durations = guest_durations.reset_index()
guest_durations = guest_durations.sort_values(by='duration', ascending=False)
# guest_durations = pd.DataFrame({'duration' : split_hosts.groupby( ['podcast', 'guests'] ).sum()}).reset_index()
guest_durations

,hosts,guests,duration
10984,Joe Rogan,Brian Redban,2702145
11569,Joey Diaz,Lee Syatt,1144116
21512,Tony Hinchcliffe,Pat Regan,532567
2532,Brian Redban,Pat Regan,532567
21493,Tony Hinchcliffe,Josh Martin,515083
2515,Brian Redban,Josh Martin,515083
7131,Doug Stanhope,Greg Chaille,514973
21483,Tony Hinchcliffe,Jeremiah Watkins,514592
2505,Brian Redban,Jeremiah Watkins,510588
21490,Tony Hinchcliffe,Joel Jimenez,503881


In [479]:
guest_durations = pd.read_csv('../reading_and_cleaning/guest_durations.csv', sep='\t', index_col=0)
G1 = nx.from_pandas_dataframe(guest_durations, 'guests', 'hosts', edge_attr=['duration'], create_using=nx.DiGraph())
#print(G1.edges(data=True)[1])


pr = nx.pagerank(G1, weight='duration')
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
save_obj(pr, 'pr_dict')

import operator

sorted_pr = sorted(pr.items(), key=operator.itemgetter(1), reverse=True)
sorted_pr


# top_ten = [i[0] for i in sorted_pr][0:10]
# top_ten

/Users/jcbrooks/anaconda3/lib/python3.5/site-packages/networkx/generators/stochastic.py:56: UserWarning:

zero out-degree for node Chris Evert

/Users/jcbrooks/anaconda3/lib/python3.5/site-packages/networkx/generators/stochastic.py:56: UserWarning:

zero out-degree for node John Calipari

/Users/jcbrooks/anaconda3/lib/python3.5/site-packages/networkx/generators/stochastic.py:56: UserWarning:

zero out-degree for node Eden Dranger

/Users/jcbrooks/anaconda3/lib/python3.5/site-packages/networkx/generators/stochastic.py:56: UserWarning:

zero out-degree for node Justine Marino

/Users/jcbrooks/anaconda3/lib/python3.5/site-packages/networkx/generators/stochastic.py:56: UserWarning:

zero out-degree for node Peter Segal



[('Joe Rogan', 0.12854148904859822),
 ('Duncan Trussell', 0.04636453571554697),
 ('Ari Shaffir', 0.02889522674558212),
 ('Chris Hardwick', 0.02572394572130148),
 ('Jesse Thorn', 0.02097267541384163),
 ('Joey Diaz', 0.02077293087479461),
 ('Alison Rosen', 0.02067690632823809),
 ('Bert Kreischer', 0.019270326402677646),
 ('Jordan Morris', 0.017597603943034626),
 ('Christopher Ryan', 0.016796336833985327),
 ('Pete Holmes', 0.016420177370568483),
 ('Kumail Nanjiani', 0.015459642393484086),
 ('Emily V. Gordon', 0.011637059983304021),
 ('Tom Segura', 0.010248087960433881),
 ('Christina Pazsitzky', 0.010248087960433881),
 ('Rhea Butcher', 0.009680447063533259),
 ('Cameron Esposito', 0.009680447063533259),
 ('Doug Benson', 0.009538018655640107),
 ('Tony Hinchcliffe', 0.009137928182092019),
 ('Brian Redban', 0.008881340511948783),
 ('Bill Maher', 0.0076202015789804374),
 ('Jay Larson', 0.007328848377389554),
 ('Ryan Sickler', 0.007328848377389554),
 ('James Altucher', 0.007326774151557891),
 ('

In [ ]:
for i in top_ten:
    print(i)
    print(split_hosts[split_hosts['hosts']==i]['date'].iloc[0])

In [480]:
hubs, authorities = nx.hits(G1)


In [487]:
sorted_hubs = sorted(hubs.items(), key=operator.itemgetter(1), reverse=True)
sorted_hubs = pd.DataFrame(sorted_hubs)

In [488]:
sorted_authorities = sorted(authorities.items(), key=operator.itemgetter(1), reverse=True)
sorted_authorities = pd.DataFrame(sorted_authorities)

In [23]:
G2 = nx.from_pandas_dataframe(guest_durations, 'guests', 'hosts', edge_attr=['duration'], create_using=nx.Graph())


In [100]:
nx.center(G2)

['Jackie Kashian',
 'Kumail Nanjiani',
 'Bert Kreischer',
 'Duncan Trussell',
 'Jesse Thorn',
 'Moshe Kasher',
 'Cameron Esposito',
 'Tim Ferriss',
 'Pete Holmes',
 'Alison Rosen',
 'Neal Brennan',
 'Margaret Cho',
 'Myq Kaplan']

In [101]:
nx.periphery(G2)

['Ruenactments',
 'Queen',
 'Paul Finebaum',
 'Freda Payne',
 'Meditation',
 'Suroosh Alvi',
 'Rachel Platten',
 'Part 1',
 'Jefferson Bethke',
 'R.C. Maxwell',
 'Yoram Bauman',
 'Philip Levine',
 'Tom Haberstroh',
 'Mallory Edens',
 'Nail Art',
 'Julie Borowski',
 'recently departed ESPNer Tom Haberstroh',
 'Rebecca Harding',
 'George Magnus',
 'Siedah Garrett',
 'Twitch Boss',
 'Graham Brownlow',
 'Maureen',
 'the head coach',
 'Make Mistakes',
 'Mary Carillo',
 'Graham Norton',
 'Jon Levy',
 'Frances Davis',
 'Jared Dudley',
 'Positive Mind',
 'Stephanie Ruhle',
 'Peita Diamantidis',
 'Trey Kerby',
 'Mariah Balenciaga',
 'Randy Hetrick',
 'Morning Routine',
 'Tommy Sotomayor',
 'Titus Burgess',
 'Dr. Chris Donaghue',
 'Milo',
 'Ted Cruz',
 'Novak Djokovic',
 'Arthur Blank',
 'Robbie Butler',
 'Oliver Payne',
 'Jonathan Fields',
 'Jane Wurwand',
 'Helen Zaltman',
 'Charles Barkley',
 'NBA confidant Carl Lentz',
 'Mike Huckabee',
 'Leah Bell',
 'Eileen Fisher',
 'Van Hunt',
 'Chi Chi 

In [102]:
sorted(nx.eccentricity(G2).items(), key=operator.itemgetter(1), reverse=False)

[('Jackie Kashian', 4),
 ('Kumail Nanjiani', 4),
 ('Bert Kreischer', 4),
 ('Duncan Trussell', 4),
 ('Jesse Thorn', 4),
 ('Moshe Kasher', 4),
 ('Cameron Esposito', 4),
 ('Tim Ferriss', 4),
 ('Pete Holmes', 4),
 ('Alison Rosen', 4),
 ('Neal Brennan', 4),
 ('Margaret Cho', 4),
 ('Myq Kaplan', 4),
 ('Dwayne Kennedy', 5),
 ('Richard Bain', 5),
 ('Felipe Esparza', 5),
 ('Jeff Chang', 5),
 ('Joe Kilgallon', 5),
 ('Christina Pazsitzky', 5),
 ('Natasha Leggero', 5),
 ('Alex Grey', 5),
 ('C.J. Toledano', 5),
 ('Dan Deacon', 5),
 ('Soman Chainani', 5),
 ('Margaret Wappler', 5),
 ('Sam Tripoli', 5),
 ('June Diane Raphael', 5),
 ('Scott Aukerman', 5),
 ('Richard Bain Telethon', 5),
 ('Stu Stone', 5),
 ('Taylor Tomlinson', 5),
 ('Henry Rollins', 5),
 ('Gina Yashere', 5),
 ('Honus From Man Man/Mister Heavenly', 5),
 ('John Bush', 5),
 ('Andrew Sleighter', 5),
 ('Alton Brown', 5),
 ('Johnny Pemberton', 5),
 ('Dave Kloc', 5),
 ('Brian Babylon', 5),
 ('Mike Lebovitz', 5),
 ('Jim Festante', 5),
 ('Luka J

In [489]:
sorted_close = sorted(nx.closeness_centrality(G2).items(), key=operator.itemgetter(1), reverse=True)
sorted_close = pd.DataFrame(sorted_close)

In [490]:
sorted_bt = sorted(nx.betweenness_centrality(G2).items(), key=operator.itemgetter(1), reverse=True)
sorted_bt = pd.DataFrame(sorted_bt)

In [491]:
sorted_degree = sorted(nx.degree_centrality(G2).items(), key=operator.itemgetter(1), reverse=True)
sorted_degree = pd.DataFrame(sorted_degree)

In [106]:
sorted(nx.eigenvector_centrality(G2).items(), key=operator.itemgetter(1), reverse=True)

[('Cameron Esposito', 0.22836049634698294),
 ('Rhea Butcher', 0.22358347863619965),
 ('Pete Holmes', 0.21883074638552671),
 ('Chris Hardwick', 0.1988964791358142),
 ('Jesse Thorn', 0.164443924548281),
 ('Jay Larson', 0.16180691691391166),
 ('Alison Rosen', 0.15019822989411152),
 ('Bert Kreischer', 0.1468441359858323),
 ('Ryan Sickler', 0.14596571122573201),
 ('Randy Sklar', 0.13863289163465473),
 ('Jason Sklar', 0.1361777179203563),
 ('Ari Shaffir', 0.13515950456422893),
 ('Jordan Morris', 0.13492536919811932),
 ('Joe Rogan', 0.13309963796042804),
 ('Doug Benson', 0.13114412634743652),
 ('John Roy', 0.11236271264745908),
 ('Todd Glass', 0.10170811368100836),
 ('Kumail Nanjiani', 0.10059597105395966),
 ('Dan St. Germain', 0.10023344002170591),
 ('Kurt Braunohler', 0.09631714471083096),
 ('Moshe Kasher', 0.09146069725763313),
 ('Sean Donnelly', 0.0911855150254345),
 ('Paul F. Tompkins', 0.08695594992943986),
 ('Shane Mauss', 0.08429806485755686),
 ('Tony Hinchcliffe', 0.0814144357409524)

In [107]:
sorted(nx.closeness_vitality(G2).items(), key=operator.itemgetter(1), reverse=True)

KeyboardInterrupt: 

In [ ]:
node_attr = [pr, hubs, authorities]
nodes_df = pd.DataFrame.from_dict(pr, orient='index')
nodes_df.rename(columns = {0:'pr'}, inplace = True)
nodes_df['hub'] = hubs.values()
nodes_df['auth'] = authorities.values()
nodes_df['eccentricity'] = nx.eccentricity(G2).values()
nodes_df['closeness'] = nx.closeness_centrality(G2).values()
nodes_df['betweenness'] = nx.betweenness_centrality(G2).values()
nodes_df['degree_cen'] = nx.degree_centrality(G2).values()
nodes_df['eigen'] = nx.eigenvector_centrality(G2).values()
nodes_df

In [ ]:
podcast_similarities = pd.DataFrame(columns=['podcast1', 'podcast2', 'score'])

for index1, row1 in podcast_info.iterrows():
    summ=0
    df1 = df[df['podcast'] == row1['Podcast Name']]
    guest_durations1 = df1.groupby(['guests'])['duration'].sum()
    guest_durations1 = guest_durations1.reset_index()
    for index2, row2 in podcast_info.iterrows():
        summ=0
        if(index1 >= index2):
            continue
        df2 = df[df['podcast'] == row2['Podcast Name']]
        guest_durations2 = df2.groupby(['guests'])['duration'].sum()
        guest_durations2 = guest_durations2.reset_index()
        for index3, row3 in guest_durations1.iterrows():
            for index4, row4 in guest_durations2.iterrows():
                if(row3['guests'] == row4['guests']):
                    summ += row3['duration']*row4['duration']
        print(row1['Podcast Name'], row2['Podcast Name'], summ)
        index3 = index1+index2
        podcast_similarities.loc[index3] = [row1['Podcast Name'], row2['Podcast Name'], summ]


podcast_similarities.to_csv('podcast_similarities.csv', sep='\t')

In [ ]:
import datetime as dt
import time

testdate = dt.datetime(2008, 1, 1, 0, 0, 1)
#print(testdate < df['date'][0])
#dt.datetime.strptime(df['date'][0], '%Y-%m-%d %H:%M:%S')
#testdate < dt.datetime.strptime(df['date'][0], '%Y-%m-%d %H:%M:%S')

#base = dt.datetime.today()
# base = dt.datetime(2018, 4, 1)
# base
#date_list = [base - dt.monthdelta(x) for x in range(0, 144)]
#date_list = [dt.datetime(2008+x/12-x%12, x%12, 1) for x in range(0, 100)]



#dt.datetime.strptime(df['date'][0], '%Y-%m-%d %H:%M:%S')

# print(testdate > date_list[0])


date_list = [dt.datetime(int(2010+(x/12)-(x%12)/12), x%12+1, 1) for x in range(0, 101)]
date_list

In [ ]:
dates = [(dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) for x in df['date']]


valid_dates = [(d < dt.datetime(2012, 1, 1)) for d in dates]
df1 = df[valid_dates]

# df1 = df[(d < dt.datetime(2012, 1, 1)) for d in dates]
len(df1)

split_hosts1 = splitDataFrameList(df1, 'hosts', ', ')
guest_durations1 = split_hosts1.groupby(['hosts', 'guests'])['duration'].sum()
guest_durations1 = guest_durations1.reset_index()

G1_1 = nx.from_pandas_dataframe(guest_durations1, 'guests', 'hosts', edge_attr=['duration'], create_using=nx.DiGraph())
pr1 = nx.pagerank(G1_1, weight='duration')

In [ ]:
pr1['Joe Rogan']

In [ ]:
dates = [(dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) for x in split_hosts['date']]

for date in date_list:
    valid_dates = [(d < date) for d in dates]
    df1 = split_hosts[valid_dates]
    #split_hosts1 = splitDataFrameList(df1, 'hosts', ', ')
    
    guest_durations1 = df1.groupby(['hosts', 'guests'])['duration'].sum()
    guest_durations1 = guest_durations1.reset_index()
    G1_1 = nx.from_pandas_dataframe(guest_durations1, 'guests', 'hosts', edge_attr=['duration'], create_using=nx.DiGraph())
    pr1 = nx.pagerank(G1_1, weight='duration')
    print(date, ' - ', pr1['Joe Rogan'])

In [6]:
import ast

df1 = df
for index1, row1 in podcast_info.iterrows():
    #df1 = df[df['podcast'] == row1['Podcast Name']].copy()
    #print(df1.is_copy)
    hosts = ast.literal_eval(row1['Hosts'])
    #print(hosts[0])
    for host in hosts:
        print(host)
        for index2, row2 in df1.iterrows():
            if(row2['guests'] == host):
                df1.drop(index=index2, inplace=True)

Joe Rogan
Duncan Trussell
Bert Kreischer
Brendan Schaub
Bryan Callen
Ari Shaffir
Russell Brand
Kevin Pereira
Chris Hardwick
Sam Harris
Tony Hinchcliffe
Brian Redban
Dave Rubin
Scott Aukerman
Ethan Klein
Marc Maron
Joey Diaz
Tom Segura
Christina Pazsitzky
Dan Harmon
Stefan Molyneux
Russ Roberts
Bill Maher
Pete Holmes
Anna Faris
Dax Shepard
Greg Fitzsimmons
Sam Tripoli
Alison Rosen
Christopher Ryan
Paul Scheer
June Diane Raphael
Jason Mantzoukas
Todd Glass
Jason Sklar
Randy Sklar
Doug Benson
Doug Benson
Howard Kremer
Kulap Vilaysack
Kumail Nanjiani
Emily V. Gordon
Graham Elwood
Chris Mancini
Neal Brennan
Moshe Kasher
Julian McCullough
Chris Cubas
Thomas Thakkar
Tommy McNamara
Iliza Shlesinger
Kurt Metzger
Sherrod Small
Todd Barry
Dan St. Germain
Sean Donnelly
Conner Moore
Steven Crowder
Nick Wiger
Mike Mitchell
Bill Simmons
Big Jay Oakerson
Luis J. Gomez
Dave Smith
Ari Shaffir
Sam Tripoli
Jayson Thibault
Hannibal Buress
Tait Fletcher
Steve Rannazzisi
Jim Rome
Jason Sklar
Randy Sklar
Bill

In [268]:
df1 = pd.read_csv('../reading_and_cleaning/guest_host_cleaned_podcasts.csv', sep='\t', index_col=0)

df1['attr'] = 'guest'

G3 = nx.from_pandas_dataframe(df1, 'guests', 'podcast', edge_attr=['date', 'duration', 'attr'], create_using=nx.Graph())

split_hosts = pd.read_csv('../reading_and_cleaning/split_hosts.csv', sep='\t', index_col=0)
G1 = nx.from_pandas_dataframe(split_hosts, 'guests', 'hosts', edge_attr=['date', 'duration'], create_using=nx.Graph())


#hosts = ast.literal_eval(row1['Hosts'])
podcast_info_split = splitDataFrameList(podcast_info, 'Hosts', ', ')
podcast_info_split['Hosts'] = podcast_info_split['Hosts'] = [g.rstrip("'") for g in podcast_info_split['Hosts']]
podcast_info_split['Hosts'] = podcast_info_split['Hosts'] = [g.rstrip('"') for g in podcast_info_split['Hosts']]
podcast_info_split['Hosts'] = podcast_info_split['Hosts'] = [g.rstrip(']') for g in podcast_info_split['Hosts']]
podcast_info_split['Hosts'] = podcast_info_split['Hosts'] = [g.rstrip("'") for g in podcast_info_split['Hosts']]
podcast_info_split['Hosts'] = podcast_info_split['Hosts'] = [g.lstrip('"') for g in podcast_info_split['Hosts']]
podcast_info_split['Hosts'] = podcast_info_split['Hosts'] = [g.lstrip('[') for g in podcast_info_split['Hosts']]
podcast_info_split['Hosts'] = podcast_info_split['Hosts'] = [g.lstrip("'") for g in podcast_info_split['Hosts']]

podcast_info_split['attr'] = 'host'
G4 = nx.from_pandas_dataframe(podcast_info_split, 'Podcast Name', 'Hosts', edge_attr=['attr'], create_using=nx.Graph())



G3.add_edges_from(G4.edges(data=True))

In [282]:
from nltk import jaccard_distance

def ngrams_split(lst, n):
    counts = dict()
    grams = [''.join(lst[i:i+n]) for i in range(len(lst)-n+1)]
    for gram in grams:
        if gram not in counts:
            counts[gram] = 1
        else:
            counts[gram] += 1
    return set(grams)

def word_search(target, words):
    low_score = 100
    prediction = ''
    words = [w for w in words if w[0]==target[0]]
    for i in range(len(words)):
        #test_word = list(words[i])
        score = jaccard_distance(ngrams_split(target,3), ngrams_split(words[i],3))
        if score<low_score:
            low_score=score
            #print(low_score)
            prediction = words[i]
            #print(words[i])
    return prediction#, low_score

correct_spellings = list(G1.nodes())
correct_spellings


print('Nick Thun' not in correct_spellings)

#word_search('Nic Thun', correct_spellings)
correct_spellings

True


['Jason Biggs',
 'Marcos Luevanos',
 'Chris Messina',
 'Brigitte Gabriel',
 'John Early',
 'Marina Krakovsky',
 'Cory n Chad The Smash Brothers',
 'Dr. Cornel West',
 'Rachel Ratner',
 'Stephanie Ruhle',
 'Jessica McKenna',
 'Alanis Morissette',
 'David J. Peterson',
 'Rep. Tim Ryan',
 'Steve Simeone',
 'Melissa Etheridge',
 'Nick Swardson',
 'Allison Scagliotti',
 'Gangnam Style',
 'TV w',
 'Rob Dyrdek',
 'Tegan',
 'Lisa Friedrich',
 'Robert Barro',
 'Wendy Simmons',
 'Tim Kennedy',
 'Christina Sommers',
 'Cathy Lynn Grossman',
 'Marc Jacobs',
 'Jameela The Transchiatrist',
 'Mahershala Ali',
 'Jared Thompson',
 'George Shapiro',
 'Jamie Denbo',
 'Matt Braunger',
 'Peter Higgs',
 'Edmund Phelps',
 'Dr Gavin Schmidt',
 'Curtis Armstrong',
 'Jon Christofaris',
 'Auggie Smith',
 'Thomas Dolby',
 'Tait Fletcher',
 'Matt Newell',
 'Bobby Lee',
 'Max Holloway',
 'Nate Corddry',
 'Dr. Rachel Armstrong',
 'Brian Henson',
 'Michael Rapaport',
 'Gerry Duggan',
 'Giorgio A. Tsoukalos',
 'Applica

In [275]:
# node1 = 'Russ Roberts'
# node2 = 'Joe Rogan'
# path = nx.shortest_path(G3, node1, node2)
# path_length = nx.shortest_path_length(G3, node1, node2)
#print(node1 + ' was a guest on ' + path[1] + ' who also had as a guest ' + node2 + '.')
#print(node1 + ' is a host of ' + path[1] + ', who had as a guest ' + path[2] + ', who was also a guest on ' + path[3] + ', who also had as a guest ' + node2 + '.')
# path

correct_spellings = list(G1.nodes())

def six_degrees(node1, node2):
    
    if(node1 not in correct_spellings):
        suggestion = word_search(node1, correct_spellings)
        message = "Sorry, we couldn't find " + node1 + " in our database. Did you mean " + suggestion + "?"
    elif (node2 not in correct_spellings):
        suggestion = word_search(node2, correct_spellings)
        message = "Sorry, we couldn't find " + node2 + " in our database. Did you mean " + suggestion + "?"
    else:
    
        path = nx.shortest_path(G3, node1, node2)
        path_length = nx.shortest_path_length(G3, node1, node2)
        message = node1
        #message = '<a target="_blank" href="">' + node1 + '</a>'
        
        for step in range(path_length+1):
            print(step, path[step])
            if(step==0):
                continue
            if(step==1):
                if(G3[node1][path[1]]['attr'] == 'host'):
                    message += ' is a host of '
                else:
                    message += ' was a guest on '
                
                message += path[1]
                #message += '<a target="_blank" href="">' + path[1] + '</a>'
                continue
            if(step % 2 == 0):
                if(G3[path[step-1]][path[step]]['attr']=='guest'):
                    if(step==2):
                        message += ', who had as a guest ' + path[step]
                        #message += ', who had as a guest ' + '<a target="_blank" href="">' + path[step] + '</a>'
                    else:
                        message += ', who also had as a guest ' + path[step]
                        #message += ', who also had as a guest ' + '<a target="_blank" href="">' + path[step] + '</a>'
                if(G3[path[step-1]][path[step]]['attr']=='host'):
                    message += ', which is hosted by ' + path[step]
                    #message += ', which is hosted by ' + '<a target="_blank" href="">' + path[step] + '</a>'
            if(step % 2 == 1):
                message += ', who was a guest on ' + path[step]
                #message += ', who was a guest on ' + '<a target="_blank" href="">' + path[step] + '</a>'

    return message
                
six_degrees('Pres. Barack Obama', 'Sam Harris')

"Sorry, we couldn't find Pres. Barack Obama in our database. Did you mean President Barack Obama?"

In [667]:
six_degrees("Snoop Dogg", 'Jenny Slate')

0 Snoop Dogg
1 Snoop Dogg's GGN Podcast
2 Jenny Slate


"Snoop Dogg is a host of Snoop Dogg's GGN Podcast, who had as a guest Jenny Slate"

In [288]:
host_list = []
for index1, row1 in podcast_info.iterrows():
    hosts = ast.literal_eval(row1['Hosts'])
    for host in hosts:
        host_list.append(host)

host_list = set(host_list)

In [504]:
import pickle
import json

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

top_host_podcast = load_obj('top_host_podcast')
top_guest_podcast = load_obj('top_guest_podcast')

host_podcasts = load_obj('host_podcasts')
guest_podcasts = load_obj('guest_podcasts')

guest_list = list(top_guest_podcast.keys())

top_ten_pr = [i[0] for i in sorted_pr][0:10]
top_fifty_pr = [i[0] for i in sorted_pr][0:50]

top_ten = pd.DataFrame(columns=['name', 'host_bool', 'host_podcast', 'guest_podcast'])
top_ten['name'] = top_ten_pr

top_fifty = pd.DataFrame(columns=['name', 'host_podcast', 'host_podcasts', 
                                  'guest_podcast', 'guest_podcasts', 
                                  'pr_rank', 'hub_rank', 'auth_rank',
                                  'degree_rank', 'bt_rank', 'close_rank'])
top_fifty['name'] = top_fifty_pr

for index, row in top_fifty.iterrows():
    row['pr_rank'] = index+1
    for index1, value1 in sorted_hubs[0].iteritems():
        if(value1==row['name']):
            row['hub_rank'] = index1+1
    for index1, value1 in sorted_authorities[0].iteritems():
        if(value1==row['name']):
            row['auth_rank'] = index1+1
    for index1, value1 in sorted_degree[0].iteritems():
        if(value1==row['name']):
            row['degree_rank'] = index1+1
    for index1, value1 in sorted_close[0].iteritems():
        if(value1==row['name']):
            row['close_rank'] = index1+1
    for index1, value1 in sorted_bt[0].iteritems():
        if(value1==row['name']):
            row['bt_rank'] = index1+1
    if(row['name'] in host_list):
        row['host_podcast'] = top_host_podcast[row['name']]
        row['host_podcasts'] = list(host_podcasts[row['name']])
    else:
        row['host_podcast'] = ''
        row['host_podcasts'] = ''
    if(row['name'] in guest_list):
        row['guest_podcast'] = top_guest_podcast[row['name']]
        row['guest_podcasts'] = list(guest_podcasts[row['name']])
    else:
        row['guest_podcast'] = ''
        row['guest_podcasts'] = ''


dict_list = top_fifty.to_dict(orient='records')
dict_list[1]

dj_list=[]
for i in range(50):
    d = {}
    d['model'] = 'rankings.People'
    d['pk'] = i
    d['fields'] = dict_list[i]
    dj_list.append(d)

    
dj_list


import json
with open('top_fifty_guest_host.json', 'w') as outfile:
    json.dump(dj_list, outfile)





In [614]:
podcast_info = pd.read_csv('../reading_and_cleaning/meta_podcast_info.csv', sep='\t', index_col=0)

p_info  = podcast_info.drop(['feedURL', 'keywords', 'cleaner'], axis=1)
p_info.columns=['name', 'hosts', 'imgurl', 'categories']

p_info['podcast_id'] = podcast_info.index-1

dict_list = p_info.to_dict(orient='records')

dj_podcasts=[]
for i in range(len(p_info)):
    d={}
    d['model'] = 'podcasts.Podcasts'
    d['pk'] = i
    d['fields'] = dict_list[i]
    dj_podcasts.append(d)
    
dj_podcasts

import json
with open('podcast_info.json', 'w') as outfile:
    json.dump(dj_podcasts, outfile)

In [466]:
sorted_close = pd.DataFrame(sorted_close)


In [603]:
from datetime import datetime, timedelta

def sec_to_hours(seconds):
    sec = timedelta(seconds=seconds)
    d = datetime(1,1,1) + sec
    
    if (seconds == 2702145):
        return ('31:6:35:45')
    else:
        return("%02d:%02d:%02d:%02d" % (d.day-1, d.hour, d.minute, d.second))
    
sec_to_hours(27021)

'00:07:30:21'

In [652]:
# df1 = pd.read_csv('../reading_and_cleaning/guest_host_cleaned_podcasts.csv', sep='\t', index_col=0)

# guest_durations_podcast = df1.groupby(['podcast', 'podcast_id', 'guests'])['duration'].sum()
# guest_durations_podcast = guest_durations_podcast.reset_index()


guest_durations_podcast = pd.read_csv('../reading_and_cleaning/guest_durations_podcast.csv', sep='\t', index_col=0)
# guest_durations_podcast.sort_values(by='duration', ascending=False, inplace=True)

len(guest_durations_podcast)

# guest_durations_podcast = guest_durations_podcast[guest_durations_podcast['duration']>600]

guest_durations_podcast['hours'] = ''
for index, row in guest_durations_podcast.iterrows():
    #print(row['duration'], sec_to_hours(row['duration']))
    guest_durations_podcast.at[index, 'hours'] = sec_to_hours(row['duration'])



len(guest_durations_podcast)
guest_durations_podcast

,podcast,podcast_id,guests,duration,hours
0,10% Happier with Dan Harris,141,Adam Levin,1937,00:00:32:17
1,10% Happier with Dan Harris,141,Adam Shankman,2538,00:00:42:18
2,10% Happier with Dan Harris,141,Alan Cumming,2102,00:00:35:02
3,10% Happier with Dan Harris,141,Ali Smith,2931,00:00:48:51
4,10% Happier with Dan Harris,141,Amy Cuddy,3319,00:00:55:19
5,10% Happier with Dan Harris,141,Anderson Cooper,2443,00:00:40:43
6,10% Happier with Dan Harris,141,Andrea Petersen,3438,00:00:57:18
7,10% Happier with Dan Harris,141,Andrew Olendzki,3763,00:01:02:43
8,10% Happier with Dan Harris,141,Andrew Scheffer,3684,00:01:01:24
9,10% Happier with Dan Harris,141,Annaka Harris,3338,00:00:55:38


In [653]:
from datetime import datetime as dt

now = dt.now()

df1 = pd.read_csv('../reading_and_cleaning/guest_host_cleaned_podcasts.csv', sep='\t', index_col=0)

dates = [(dt.strptime(x, '%Y-%m-%d %H:%M:%S')) for x in df1['date']]
sec_delta = [(now-date).total_seconds() for date in dates]
sec_delta
df1['sec_delta'] = sec_delta
df1

guest_recent_date = df1.groupby(['podcast', 'podcast_id', 'guests']).agg({'sec_delta':'min', 'date':'first'})   #['date', 'sec_delta'].min()
guest_recent_date = guest_recent_date.reset_index()
guest_recent_date[guest_recent_date['podcast']=='The Joe Rogan Experience']


,podcast,podcast_id,guests,date,sec_delta
13173,The Joe Rogan Experience,0,"""Big"" Jay Oakerson",2015-05-05 01:57:54,9.500833e+07
13174,The Joe Rogan Experience,0,Abby Martin,2017-04-26 00:52:57,3.263143e+07
13175,The Joe Rogan Experience,0,Action Bronson,2016-02-23 21:19:05,6.953706e+07
13176,The Joe Rogan Experience,0,Adam Carolla,2015-03-10 01:04:37,9.984993e+07
13177,The Joe Rogan Experience,0,Adam Cropp,2016-03-18 23:26:32,6.745581e+07
13178,The Joe Rogan Experience,0,Adam Greentree,2017-10-26 02:42:09,1.681367e+07
13179,The Joe Rogan Experience,0,Adam Hunter,2013-01-09 07:53:13,1.680814e+08
13180,The Joe Rogan Experience,0,Adam Kokesh,2012-07-12 20:36:27,1.836740e+08
13181,The Joe Rogan Experience,0,Adam Lowery,2015-07-23 01:36:28,8.818401e+07
13182,The Joe Rogan Experience,0,Adam Scorgie,2015-05-15 00:16:16,9.415043e+07


In [654]:
guest_durations_podcast['recent'] = guest_recent_date['date']

guest_durations_podcast.sort_values(by='duration', ascending=False, inplace=True)
guest_durations_podcast.drop(columns='duration', inplace=True)
guest_durations_podcast
# guest_durations_podcast['recent'] = [(dt.strptime(x, '%Y-%m-%d %H:%M:%S')).date() for x in guest_durations_podcast['recent']]
# guest_durations_podcast['recent'][0]

,podcast,podcast_id,guests,hours,recent
13237,The Joe Rogan Experience,0,Brian Redban,31:6:35:45,2018-03-20 22:29:51
11783,The Church of What's Happening Now: With Joey ...,14,Lee Syatt,13:05:48:36,2018-02-06 09:24:59
6477,Kill Tony,9,Pat Regan,06:03:56:07,2018-03-30 02:44:37
6460,Kill Tony,9,Josh Martin,05:23:04:43,2018-03-30 02:44:37
12261,The Doug Stanhope Podcast,126,Greg Chaille,05:23:02:53,2018-03-30 10:15:00
6450,Kill Tony,9,Jeremiah Watkins,05:21:49:48,2018-03-30 02:44:37
6457,Kill Tony,9,Joel Jimenez,05:19:58:01,2018-03-30 02:44:37
13245,The Joe Rogan Experience,0,Bryan Callen,04:22:42:46,2018-03-07 03:06:07
13440,The Joe Rogan Experience,0,Joey Diaz,04:21:07:32,2018-02-01 23:30:19
12679,The Full Charge Power Hour featuring Matt Fulc...,116,Craig Coleman,04:20:09:51,2018-03-12 19:36:49


In [655]:
dict_list = guest_durations_podcast.to_dict(orient='records')


gdp_dj=[]
for i in range(len(guest_durations_podcast)):
    d = {}
    d['model'] = 'rankings.Durations'
    d['pk'] = i
    d['fields'] = dict_list[i]
    gdp_dj.append(d)

    
gdp_dj


import json
with open('guest_duration_podcast.json', 'w') as outfile:
    json.dump(gdp_dj, outfile)

In [615]:
podcast_similarities = pd.read_csv('podcast_similarities.csv', sep='\t', index_col=0)
podcast_similarities.sort_values(by='score', ascending=False, inplace=True)
podcast_similarities = podcast_similarities[podcast_similarities['score']>0]
podcast_similarities

similarities = pd.DataFrame(columns=['podcast1', 'podcast2', 'podcast2_id', 'score'])

num = len(podcast_info)
index=0
for index2, row2 in podcast_info.iterrows():
    df1 = podcast_similarities[podcast_similarities['podcast1']==row2['Podcast Name']].copy()
    df2 = podcast_similarities[podcast_similarities['podcast2']==row2['Podcast Name']].copy()
    df = pd.concat([df1, df2], ignore_index=True)
#     df.sort_values(by='score', ascending=False, inplace=True)
    for index1, row1 in df.iterrows():
        if(index1>7):
            continue
        if(row1['podcast1']==row2['Podcast Name']):
            podcast2 = row1['podcast2']
            
        else:
            podcast2 = row1['podcast1']
        for index3, row3 in podcast_info.iterrows():
            if(row3['Podcast Name']==podcast2):
                podcast2_id = index3-1
        index+=1
        if(row1['score']>0):
            similarities.loc[index] = [row2['Podcast Name'], podcast2, podcast2_id, row1['score']]
        
similarities

,podcast1,podcast2,podcast2_id,score
1,The Joe Rogan Experience,The Church of What's Happening Now: With Joey ...,14,164983535755
2,The Joe Rogan Experience,Bertcast's podcast,2,110151487064
3,The Joe Rogan Experience,The Duncan Trussell Family Hour,1,70508807688
4,The Joe Rogan Experience,Ari Shaffir's Skeptic Tank,4,70070591436
5,The Joe Rogan Experience,Your Mom's House with Christina P. and Tom Segura,15,56556176794
6,The Joe Rogan Experience,Alison Rosen Is Your New Best Friend,25,38797081576
7,The Joe Rogan Experience,Tin Foil Hat With Sam Tripoli,24,38352264748
8,The Joe Rogan Experience,Punch Drunk Sports,48,37895839560
9,The Duncan Trussell Family Hour,The Bone Zone,81,13152814504
10,The Duncan Trussell Family Hour,Ari Shaffir's Skeptic Tank,4,10165982695


In [616]:
dict_list = similarities.to_dict(orient='records')


sim_dj=[]
for i in range(len(dict_list)):
    d = {}
    d['model'] = 'podcasts.Similar'
    d['pk'] = i
    d['fields'] = dict_list[i]
    sim_dj.append(d)

    
sim_dj


import json
with open('similarities.json', 'w') as outfile:
    json.dump(sim_dj, outfile)